<a href="https://colab.research.google.com/github/Nilanjan-Chatterjee/Aahar/blob/main/FasterRCNNFNAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FasterRCNN FNAC**

In [ ]:
#connecting the drive to colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import cv2

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
!git clone https://github.com/harshatejas/pytorch_custom_object_detection.git
%cd /content/pytorch_custom_object_detection
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import os
import torch
import glob
from torch.utils.data import Dataset
import time
import copy
import numpy as np
import math
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from torch.utils.data.dataset import Dataset  # For custom data-sets
import torchvision.transforms as transforms
from PIL import Image
import torchvision
import matplotlib.pyplot as plt
import argparse
import logging
import sys
import cv2
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import random
from google.colab.patches import cv2_imshow
def create_label_txt(path_to_csv):

	data = pd.read_csv(path_to_csv)
	labels = data['class'].unique()

	labels_dict = {}

	# Creat dictionary from array
	for index, label in enumerate(labels):
		labels_dict.__setitem__(index, label)

	# We need to create labels.txt and write labels dictionary into it
	with open('labels.txt', 'w') as f:
		f.write(str(labels_dict))

	return labels_dict	

def parse_one_annot(path, filename, labels_dict):

	data = pd.read_csv(path)

	class_names = data['class'].unique()
	classes_df = data[data["filename"] == filename]["class"]
	classes_array = classes_df.to_numpy()
	
	boxes_df = data[data["filename"] == filename][["xmin", "ymin", "xmax", "ymax"]]
	boxes_array = boxes_df.to_numpy()
	
	classes = []
	for key, value in labels_dict.items():
		for i in classes_array:
			if i == value:
				classes.append(key)

	# Convert list to tuple
	classes = tuple(classes)

	return boxes_array, classes

def get_model(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = False)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model


def get_transforms(train):

	transforms = []

	# Convert numpy image to PyTorch Tensor
	transforms.append(T.ToTensor())

	if train:
		# Data augmentation
		transforms.append(T.RandomHorizontalFlip(0.5))

	return T.Compose(transforms)

Cloning into 'pytorch_custom_object_detection'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (441/441), done.
remote: Total 447 (delta 28), reused 380 (delta 4), pack-reused 0
Receiving objects: 100% (447/447), 36.73 MiB | 33.61 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/pytorch_custom_object_detection


In [ ]:
class ListDataset(Dataset):
    def __init__(self, img_files,label_files, img_size=448,transforms = None):
        self.transforms = transforms
        self.inputs_dtype = torch.float64
        self.targets_dtype = torch.long
        self.img_files=img_files
        self.label_files=label_files
        self.img_shape = (img_size, img_size)
        self.max_objects = 200

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):

        img_path = self.img_files[index % len(self.img_files)].rstrip()
        img = cv2.imread(img_path)
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        label_path = self.label_files[index % len(self.img_files)].rstrip()
        labels = None
        labelsz=[]
        #print("input image shape",input_img.shape)
        if os.path.exists(label_path):
            labels = np.loadtxt(label_path)
            cls=[]
            for i in range(len(labels)):
                if(labels[i][3]>0.0001 and labels[i][4]>0.0001):
                    labelsz.append(labels[i][1:])
                    cls.append(labels[i][0])
            labels=labelsz
            labels=np.array(labels)
            labels.reshape(-1, 4)
            x1 = (labels[:,0])*input_img.shape[1]#w * (labels[:, 1] - labels[:, 3]/2)
            y1 = (labels[:,1])*input_img.shape[0]#h * (labels[:, 2] - labels[:, 4]/2)
            x2 = (labels[:,0]+labels[:,2])*input_img.shape[1]#w * (labels[:, 1] + labels[:, 3]/2)
            y2 = (labels[:,1]+labels[:,3])*input_img.shape[0]#h * (labels[:, 2] + labels[:, 4]/2)
            # Calculate ratios from coordinates
            labels[:, 0] = x1 # / padded_w
            labels[:, 1] = y1# / padded_w
            labels[:, 2] = x2# / padded_h
            labels[:, 3] = y2# / padded_h
        labels = np.where(labels<0, 0, labels)
        obj=[]
        iscorwd=[]
        obj=cls
        """for i in range(len(labels)):
         
          #l.append(labels)
          #labelsx.append(labels[i][:].tolist())
          obj.append(1)
          #iscrowd.append(1)"""
        labelsx=np.array(labels)
        
        num_box=len(labels)
        # Fill matrix
        filled_labels = np.zeros((self.max_objects, 4))
        if labels is not None:
            filled_labels[range(len(labels))[:self.max_objects]] = labels[:self.max_objects]
        iscrowd = torch.zeros((num_box,), dtype=torch.int64)
        #img =img.reshape(3,1536,2048)
        labels=labelsx
        labels=labels.tolist()
        #obj=obj.tolist()
        #print(labels)
        #print(obj)
        labels=np.array(labels)
        
        obj=np.array(obj)
        #labels=labels.tolist()
        #obj=obj.tolist()
        img=img/255
              
        


        boxes = torch.as_tensor(labels, dtype = torch.float32)
        
        obj=[1.0 for i in range(len(obj))]
        #print("obj",obj)
        labels = torch.tensor(obj, dtype=torch.int64)
        #boxes*=255
        image_id = torch.tensor([index])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        iscrowd = torch.tensor(obj, dtype=torch.int64)
        
        #img=img.astype(double)
        target = {}
        #print("labels",labels)
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        #print("start list target") 
        #print(target)
        #print("end list target")
       # print(boxes)
        if self.transforms is not None:
          input_img, target = self.transforms(input_img, target)
        #print("these are boxes:",boxes)
        #print(labels)
        #print(image_id)
        #print(area)
        #print(iscrowd)
        #a=list(targets)
        #a.append(target)
        #print("image",type(input_img),"boxes:",type(boxes),",labels:",type(labels),",image id:",image_id,",area:",area,",iscrowd:",iscrowd)
        #print("targets:",target)
        #print("image:",input_img)
        return input_img,target


In [ ]:
batch_size = 1
inp_dim = 800
path='/content/gdrive/MyDrive/Data_localization-FNAC/'
image_train=glob.glob(path+'train/images/'+'*.jpg')
image_val=glob.glob(path+'val/images/'+'*.jpg')
label_files = []
inputs_dtype = torch.float32
targets_dtype = torch.long
label_train=[]
label_val=[]
for img in image_train:
    label_train.append(path+'train/text/'+img[len(path)+len('train/images/')-1:-4]+'.txt')
for img in image_val:
    label_val.append(path+'val/text/'+img[len(path)+len('val/images/')-1:-4]+'.txt')
print(len(image_val))
print(len(label_val))
train_dataset=ListDataset(image_train,label_train, img_size=inp_dim, transforms = get_transforms(train = True))
valid_dataset=ListDataset(image_val,label_val, img_size=inp_dim, transforms = get_transforms(train = False))
dataloaders_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=1,collate_fn = utils.collate_fn)
dataloaders_val = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True,num_workers=1,collate_fn = utils.collate_fn)
print(len(dataloaders_train))
for i_batch, sample in enumerate(dataloaders_train):
    #print(type(sample))
    img,targets=sample
    #print("image")
    #print(img)
    print(type(img[0]))
    #print("start target")
    print(targets)
    boxes=targets[0]["boxes"]
    """for element in boxes:
      start_point=(element[0],element[1])
      end_point=(element[2],element[3])
      cv2.rectangle(img[0],start_point,end_point,(255,0,0),3)
      cv2_imshow(input_img)"""
    #cv2_imshow(img[0])  
    #print("end target")
    if i_batch == 0:
        break

12
12
50
<class 'torch.Tensor'>
({'boxes': tensor([[ 800.9997,  364.9997,  856.0000,  408.0000],
        [ 828.9997,  385.0003,  888.9997,  444.0000],
        [ 781.9994,  204.0000,  844.0000,  256.0003],
        [ 795.0003,  145.9997,  852.9997,  195.9994],
        [ 837.0010,    0.0000,  871.0003,   48.0000],
        [ 851.0003,   73.9997,  912.0000,  126.0000],
        [ 848.9997,  127.9997,  919.0003,  190.9997],
        [ 915.0003,   40.9997,  984.0000,   90.9994],
        [ 860.0000,  568.9997,  915.0003,  625.9997],
        [ 920.0000,  571.0003,  972.0000,  620.0006],
        [ 759.0003,  789.0000,  822.0006,  850.9997],
        [ 821.0010,  820.0003,  886.0006,  881.0006],
        [ 923.0003,  877.9997,  968.9997,  924.9994],
        [ 887.0003,  792.0000,  936.9997,  853.9997],
        [ 901.9994,  714.0000,  972.9997,  772.9997],
        [ 948.9997,  487.0003, 1016.9997,  563.0006],
        [1051.0004,  430.0003, 1083.0004,  462.0000],
        [ 992.9997,  379.0003, 1061.999

In [ ]:
def iou(a,b):
    ulresult=0
    count=0
    for i in a:
        for j in b:
            xmin,ymin,xmax,ymax=0,0,0,0
            if(i[0]<j[0] and i[1]<j[1] and i[2]>j[0] and i[3]>j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ulresult+=result
                    count+=1
            elif(i[1]>j[1] and i[1]<j[3] and i[2]>j[0]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ulresult+=result
                    count+=1
            elif(i[0]>j[0] and i[0]<j[2] and i[3]<j[1]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ulresult+=result
                    count+=1
            elif(i[0]>j[0] and i[1]>j[1] and i[0]<j[2] and i[1]<j[3]):
                xmin=max(i[0],j[0])
                ymin=max(i[1],j[1])
                xmax=min(i[2],j[2])
                ymax=min(i[3],j[3])
                area=(ymax - ymin) * (xmax-xmin)
                aarea=(i[2]-i[0])*(i[3]-i[1])
                barea=(j[2]-j[0])*(j[3]-j[1])
                result=area/(aarea+barea-area)
                if(result>0.5):
                    ulresult+=result
                    count+=1
    if(count==0):
        return(-1)
    else:
        print(ulresult,"      ",count)
        return(ulresult/count)
import cv2
def get_model(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model
def zaccard(data_loader_test,loaded_model):
    ulresult=0
    ulcount=0
    zzz=0
    #print(len(dataloaders_test))
    for i_batch, sample_batched in enumerate(data_loader_test):
        img,targets=sample_batched
        #print("For batch:",i_batch)
         #print(targets)
        zzz+=1
    #image = cv2.imread(img_path)
 #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 # img = torchvision.transforms.ToTensor()(img)
    #print(img)
        img=img[0]
    #img = torchvision.transforms.ToTensor()(img)
        #loaded_model = get_model(6)
        #loaded_model.load_state_dict(torch.load(output_dir + "/fasterRCNNfinalmodel"))
        img=img.to(device=device)
        #targets=targets.to(device=device)
        loaded_model.eval()
        with torch.no_grad():
            prediction = loaded_model([img])
            boxes=prediction[0]["boxes"]
       # print("boxes:",boxes)
    #print(targets[0]["boxes"])
        targets=np.array(targets[0]["boxes"])
        boxes=boxes.cpu()
        boxes=np.array(boxes)
    #print("targets",targets)
    #print("boxes",boxes)
        result=iou(targets,boxes)
        print(result)
        if(result!=-1):
            ulresult+=result
            ulcount+=1
    if(ulcount==0):
        
        return(0)
    return(ulresult/ulcount)


In [ ]:
test_set_length = 40 		 # Test set (number of images)
train_batch_size = 1  		 # Train batch size
test_batch_size = 1    		 # Test batch size
num_classes = 6       		 # Number of classes
learning_rate = 0.005  		 # Learning rate
num_epochs=5
eps=0
device='cuda'
model = get_model(num_classes)
model.to(device = device)
modelfinal=get_model(num_classes)
modelfinal.to(device = device)
zzzfinal=0
# Construct the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = learning_rate, momentum = 0.9, weight_decay = 0.0005)
output_path='/content/gdrive/MyDrive/'
# Learning rate scheduler decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)
for epoch in range(num_epochs):   
    
    train_one_epoch(model, optimizer, dataloaders_train, device, epoch, print_freq = 10)
    lr_scheduler.step()
    
    zzz=zaccard(dataloaders_val,model)
    print("Zaccard index for epoch",epoch,"is",zzz)
    if(zzzfinal<zzz):
        modelfinal=model
        zzzfinal=zzz
        eps=epoch

# Save the model state	
#torch.save(modelfinal.state_dict(), output_dir + "/fasterRCNNultimate")

torch.save(modelfinal.state_dict(), output_path + "fasterrcnnfnac.pt")

Epoch: [0]  [ 0/50]  eta: 0:00:48  lr: 0.000107  loss: 6.3959 (6.3959)  loss_classifier: 1.7553 (1.7553)  loss_box_reg: 0.7043 (0.7043)  loss_objectness: 3.8092 (3.8092)  loss_rpn_box_reg: 0.1271 (0.1271)  time: 0.9615  data: 0.1972  max mem: 7878
Epoch: [0]  [10/50]  eta: 0:00:32  lr: 0.001126  loss: 2.3886 (3.5190)  loss_classifier: 1.4388 (1.3080)  loss_box_reg: 0.6065 (0.5933)  loss_objectness: 0.5700 (1.5281)  loss_rpn_box_reg: 0.0818 (0.0896)  time: 0.8002  data: 0.0262  max mem: 7878
Epoch: [0]  [20/50]  eta: 0:00:23  lr: 0.002146  loss: 1.4313 (2.3990)  loss_classifier: 0.5283 (0.8639)  loss_box_reg: 0.5121 (0.5457)  loss_objectness: 0.2759 (0.9019)  loss_rpn_box_reg: 0.0728 (0.0875)  time: 0.7879  data: 0.0094  max mem: 7878
Epoch: [0]  [30/50]  eta: 0:00:15  lr: 0.003165  loss: 1.1373 (2.0938)  loss_classifier: 0.3449 (0.7198)  loss_box_reg: 0.4961 (0.5393)  loss_objectness: 0.2035 (0.7377)  loss_rpn_box_reg: 0.0886 (0.0970)  time: 0.7904  data: 0.0095  max mem: 8101
Epoch: [